In [1]:
# 🔒 BULLETPROOF project + config import cell

import sys
from pathlib import Path
import os

# Detect project root by searching for config.py
current = Path.cwd()

PROJECT_ROOT = None
for parent in [current] + list(current.parents):
    if (parent / "config.py").exists():
        PROJECT_ROOT = parent
        break

if PROJECT_ROOT is None:
    raise RuntimeError("❌ config.py not found in any parent directory")

# Add project root to sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Set working directory
os.chdir(PROJECT_ROOT)

# Now import config
import config

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("📦 Config loaded from:", config.__file__)




✅ PROJECT_ROOT: c:\Users\rajku\Desktop\enterprise_knowledge_assistant
📦 Config loaded from: c:\Users\rajku\Desktop\enterprise_knowledge_assistant\config.py


In [2]:
# import necessary packages

import re
from sentence_transformers import SentenceTransformer, util

from src.vector_store.faiss_store import load_vector_store
from src.rag.citation import retrieve_with_citations


In [3]:
# Load Models & Vector Store

# Embedding model (same as retrieval)
embedding_model = SentenceTransformer(config.EMBEDDING_MODEL_NAME)

# Vector store
faiss_index, documents = load_vector_store(config.VECTOR_STORE_PATH)

print(f"✅ Loaded {faiss_index.ntotal} vectors")


✅ Vector store loaded from:
C:\Users\rajku\Desktop\enterprise_knowledge_assistant\data\processed\vector_store.pkl
✅ Loaded 1797 vectors


In [13]:
# Example Inputs (From Step-6)----> Paste one real answer generated earlier.

query = "thermal behaviour"

answer = """
--------------------------------------------------------------------------------[1] (distance=1.0176)
into the mechanical behaviour under simulated operational conditions. The data illustrated in 
Figures 20 and 21 elucidate these strain rate responses throughout the thermal cycle, particularly 
noting significant inflexion points at 2,000 and 14,000 seconds within the cycle timeline.

[2] (distance=1.0396)
density increases as the temperature increases. The materials exhibit different rates of increase, with 
Sn63Pb37 having a more pronounced rise than the others. This kind of analysis is useful in selecting 
materials for thermal applications where expansion due to heat can affect performance.
"""


In [14]:
# Retrieve Context Again (Ground Truth)

retrieved_chunks = retrieve_with_citations(
    query=query,
    model=embedding_model,
    index=faiss_index,
    documents=documents,
    top_k=config.TOP_K
)

retrieved_text = " ".join([c["content"] for c in retrieved_chunks])


In [15]:
# METRIC 1 — Citation Coverage
# Question: Did the answer actually use citations?

def citation_coverage(answer: str) -> float:
    citations = re.findall(r"\[\d+\]", answer)
    return len(set(citations))


In [16]:
num_citations = citation_coverage(answer)
print("📌 Citation count:", num_citations)


📌 Citation count: 2


#### Interpretation

0 → ❌ ungrounded

>=1 → ✅ grounded answer

In [17]:
# METRIC 2 — Faithfulness Score (MOST IMPORTANT)
## Question: Does the answer content actually come from retrieved context?
## We use semantic similarity between:   answer and retrieved context

def faithfulness_score(
    answer: str,
    retrieved_context: str,
    model: SentenceTransformer
) -> float:

    emb_answer = model.encode(answer, convert_to_tensor=True)
    emb_context = model.encode(retrieved_context, convert_to_tensor=True)

    score = util.cos_sim(emb_answer, emb_context).item()
    return score


In [18]:
faith_score = faithfulness_score(answer, retrieved_text, embedding_model)
print(f"🧠 Faithfulness score: {faith_score:.3f}")


🧠 Faithfulness score: 0.782


# Interpretation (Very Important)
Score	Meaning
> 0.75	✅ Highly faithful

0.55 – 0.75	⚠ Partially faithful

< 0.55	❌ Likely hallucination

In [19]:
# Retrieval Quality
## Question: Were retrieved chunks actually relevant to the query?

## We measure similarity between: query and each retrieved chunk

def retrieval_relevance(
    query: str,
    retrieved_chunks,
    model
):
    q_emb = model.encode(query, convert_to_tensor=True)
    scores = []

    for c in retrieved_chunks:
        c_emb = model.encode(c["content"], convert_to_tensor=True)
        scores.append(util.cos_sim(q_emb, c_emb).item())

    return scores


In [20]:
relevance_scores = retrieval_relevance(
    query, retrieved_chunks, embedding_model
)

for i, score in enumerate(relevance_scores, 1):
    print(f"Chunk {i} relevance: {score:.3f}")


Chunk 1 relevance: 0.491
Chunk 2 relevance: 0.480
Chunk 3 relevance: 0.442
Chunk 4 relevance: 0.420


In [21]:
# FINAL EVALUATION SUMMARY CELL

print("===== RAG EVALUATION SUMMARY =====")
print("Query:", query)
print("Citations used:", num_citations)
print(f"Faithfulness score: {faith_score:.3f}")
print(f"Avg retrieval relevance: {sum(relevance_scores)/len(relevance_scores):.3f}")


===== RAG EVALUATION SUMMARY =====
Query: thermal behaviour
Citations used: 2
Faithfulness score: 0.782
Avg retrieval relevance: 0.458
